# Data Wrangling 1.3

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

import json

import csv

import pprint

from datetime import datetime as dt


In [2]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [3]:
cursor = connection.cursor()

# Lab: Reading Flat JSON Files

In [4]:
def my_read_flat_json(file_name, limit_lines, limit_json):
    "read a flat json file, detect if it has a header, detect if it is in big data format"
    
    print("------------------------------------")
    print("   ", file_name)
    print("------------------------------------")
    
    f = open(file_name, "r")
    
    data = f.read()
    
    f.close()
    
    lines = data.splitlines(False)
    
    i = 0
    
    for line in lines:
        print(line)
        i += 1
        if i == limit_lines:
            break
    
    print("\n>>> Printed", limit_lines, "lines of", len(lines), "total lines.")
    
    if lines[0][0] == '{' and lines[0][-1] == '}':
        json_temp = []
        for line in lines:
            json_temp.append(json.loads(line))
    else:
        json_temp = json.loads(data)
    
    if type(json_temp) == dict:
        json_list = json_temp['data']
    else:
        json_list = json_temp
    
    i = 0
    
    for j in json_list:
        print("\n>>>JSON Object #", i, "unformatted:\n\n", j)
        print("\n>>>JSON Object #", i, "pretty printed:\n")
        pprint.pprint(j, sort_dicts=False, indent=2)
        i += 1
        if i == limit_json:
            break;

    print('\n')
    

In [5]:
my_read_flat_json("temp_stores_no_header.json", 25, 3)

------------------------------------
    temp_stores_no_header.json
------------------------------------
[
  {
    "store_id": 1,
    "street": "3000 Telegraph Ave",
    "city": "Berkeley",
    "state": "CA",
    "zip": "94705",
    "latitude": 37.8555,
    "longitude": -122.2604
  },
  {
    "store_id": 2,
    "street": "1001 Broadway",
    "city": "Seattle",
    "state": "WA",
    "zip": "98122",
    "latitude": 47.6114,
    "longitude": -122.3214
  },
  {
    "store_id": 3,
    "street": "2510 McKinney Ave",
    "city": "Dallas",
    "state": "TX",
    "zip": "75201",

>>> Printed 25 lines of 47 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'longitude': -122.2604}

>>>JSON Object # 0 pretty printed:

{ 'store_id': 1,
  'street': '3000 Telegraph Ave',
  'city': 'Berkeley',
  'state': 'CA',
  'zip': '94705',
  'latitude': 37.8555,
  'longitude': -122.2604}

>>>JSON

In [6]:
my_read_flat_json("temp_stores_header.json", 25, 3)

------------------------------------
    temp_stores_header.json
------------------------------------
{
  "creator": "Acme Gourmet Meals",
  "timestamp": "2021-07-10 23:30:36",
  "file_name": "temp_stores_header.json",
  "version": "12.4.7",
  "legal": "Unauthorized use, duplication, or possession, blah, blah",
  "data": [
    {
      "store_id": 1,
      "street": "3000 Telegraph Ave",
      "city": "Berkeley",
      "state": "CA",
      "zip": "94705",
      "latitude": 37.8555,
      "longitude": -122.2604
    },
    {
      "store_id": 2,
      "street": "1001 Broadway",
      "city": "Seattle",
      "state": "WA",
      "zip": "98122",
      "latitude": 47.6114,
      "longitude": -122.3214
    },

>>> Printed 25 lines of 54 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'longitude': -122.2604}

>>>JSON Object # 0 pretty printed:

{ 'store_id': 1,
  'street': '

In [7]:
my_read_flat_json("temp_stores_big_data.json", 25, 3)

------------------------------------
    temp_stores_big_data.json
------------------------------------
{"store_id": 1, "street": "3000 Telegraph Ave", "city": "Berkeley", "state": "CA", "zip": "94705", "latitude": 37.8555, "longitude": -122.2604}
{"store_id": 2, "street": "1001 Broadway", "city": "Seattle", "state": "WA", "zip": "98122", "latitude": 47.6114, "longitude": -122.3214}
{"store_id": 3, "street": "2510 McKinney Ave", "city": "Dallas", "state": "TX", "zip": "75201", "latitude": 32.7958, "longitude": -96.8015}
{"store_id": 4, "street": "299 SE 3rd Ave", "city": "Miami", "state": "FL", "zip": "33131", "latitude": 25.772, "longitude": -80.1891}
{"store_id": 5, "street": "1202 Broadway", "city": "Nashville", "state": "TN", "zip": "37203", "latitude": 36.1568, "longitude": -86.7881}

>>> Printed 25 lines of 5 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'long

## You try it - read and understand the structure of the following json files:

* temp_sales_no_header.json
* temp_sales_header.json
* temp_sales_big_data.json

* temp_line_items_no_header.json
* temp_line_items_header.json
* temp_line_items_big_data.json

* temp_customers_no_header.json
* temp_customers_header.json
* temp_customers_big_data.json

* temp_products_no_header.json
* temp_products_header.json
* temp_products_big_data.json

* temp_holidays_no_header.json
* temp_holidays_header.json
* temp_holidays_big_data.json


# Lab: Loading Flat JSON Data into Database Tables

In [11]:
def my_flat_json_2_csv(file_name):
    "read a flat json file, convert to csv"
        
    f = open(file_name, "r")
    
    data = f.read()
    
    f.close()
    
    output_file_name = file_name[:-4] + "csv"
    
    f = open(output_file_name, "w")
    
    lines = data.splitlines(False)
    
    if lines[0][0] == '{' and lines[0][-1] == '}':
        json_temp = []
        for line in lines:
            json_temp.append(json.loads(line))
    else:
        json_temp = json.loads(data)
    
    if type(json_temp) == dict:
        json_list = json_temp['data']
    else:
        json_list = json_temp
    
    dw = csv.DictWriter(f, json_list[0].keys())
    dw.writeheader()
    dw.writerows(json_list)
    
    f.close()

In [12]:
my_flat_json_2_csv("temp_stores_no_header.json")

In [13]:
my_flat_json_2_csv("temp_stores_header.json")

In [14]:
my_flat_json_2_csv("temp_stores_big_data.json")

In [15]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [16]:
my_read_csv_file("temp_stores_no_header.csv", 25)

['store_id', 'street', 'city', 'state', 'zip', 'latitude', 'longitude']
['1', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705', '37.8555', '-122.2604']
['2', '1001 Broadway', 'Seattle', 'WA', '98122', '47.6114', '-122.3214']
['3', '2510 McKinney Ave', 'Dallas', 'TX', '75201', '32.7958', '-96.8015']
['4', '299 SE 3rd Ave', 'Miami', 'FL', '33131', '25.772', '-80.1891']
['5', '1202 Broadway', 'Nashville', 'TN', '37203', '36.1568', '-86.7881']

Printed  6 lines of  6 total lines.


In [17]:
my_read_csv_file("temp_stores_header.csv", 25)

['store_id', 'street', 'city', 'state', 'zip', 'latitude', 'longitude']
['1', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705', '37.8555', '-122.2604']
['2', '1001 Broadway', 'Seattle', 'WA', '98122', '47.6114', '-122.3214']
['3', '2510 McKinney Ave', 'Dallas', 'TX', '75201', '32.7958', '-96.8015']
['4', '299 SE 3rd Ave', 'Miami', 'FL', '33131', '25.772', '-80.1891']
['5', '1202 Broadway', 'Nashville', 'TN', '37203', '36.1568', '-86.7881']

Printed  6 lines of  6 total lines.


In [18]:
my_read_csv_file("temp_stores_big_data.csv", 25)

['store_id', 'street', 'city', 'state', 'zip', 'latitude', 'longitude']
['1', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705', '37.8555', '-122.2604']
['2', '1001 Broadway', 'Seattle', 'WA', '98122', '47.6114', '-122.3214']
['3', '2510 McKinney Ave', 'Dallas', 'TX', '75201', '32.7958', '-96.8015']
['4', '299 SE 3rd Ave', 'Miami', 'FL', '33131', '25.772', '-80.1891']
['5', '1202 Broadway', 'Nashville', 'TN', '37203', '36.1568', '-86.7881']

Printed  6 lines of  6 total lines.


## At this point, we have csv files;  we know from earlier labs how to load csv files into database tables

## You try it - convert the following flat json files to csv format;  read the csv files to verify that they are ready for loading into a database:


* temp_sales_no_header.json
* temp_sales_header.json
* temp_sales_big_data.json

* temp_line_items_no_header.json
* temp_line_items_header.json
* temp_line_items_big_data.json

* temp_customers_no_header.json
* temp_customers_header.json
* temp_customers_big_data.json

* temp_products_no_header.json
* temp_products_header.json
* temp_products_big_data.json

* temp_holidays_no_header.json
* temp_holidays_header.json
* temp_holidays_big_data.json


# Lab: Extracting Flat JSON Files

In [19]:
def my_extract_flat_json(query, file_name, file_type):
    "using the query provided, extract to json, file_type: 1 = no header, 2 = header, 3 = big_data"
    
    connection.rollback()
    
    cursor.execute(query)
    
    connection.rollback()

    rows = cursor.fetchall()
    
    list_of_json = []
    
    for row in rows:
        list_of_json.append(row[0])
        
    f = open(file_name, "w")
    
    # flat json with no headers
    if file_type == 1:
        json.dump(list_of_json, f, indent=2)
    
    # flat json with a header
    if file_type == 2:
        template = {"creator": "Acme Gourmet Meals",
                    "timestamp": dt.now().strftime("%Y-%d-%m %H:%M:%S"),
                    "file_name": file_name, 
                    "version": "12.4.7",
                    "legal": "Unauthorized use, duplication, or possession, blah, blah",
                    "data": list_of_json
                   }
        json.dump(template, f, indent=2)
        
    # flat json big data style
    if file_type == 3:
        for j in list_of_json:
            f.write(json.dumps(j) + "\n")
        
    f.close()  
    

In [21]:
query = """

select row_to_json(a) 
from (select *
      from temp_stores
      order by store_id) as a
      
"""

my_extract_flat_json(query, "temp_stores_no_header_2.json", 1)
my_extract_flat_json(query, "temp_stores_header_2.json", 2)
my_extract_flat_json(query, "temp_stores_big_data_2.json", 3)


In [22]:
my_read_flat_json("temp_stores_no_header_2.json", 25, 3)

------------------------------------
    temp_stores_no_header_2.json
------------------------------------
[
  {
    "store_id": 1,
    "street": "3000 Telegraph Ave",
    "city": "Berkeley",
    "state": "CA",
    "zip": "94705",
    "latitude": 37.8555,
    "longitude": -122.2604
  },
  {
    "store_id": 2,
    "street": "1001 Broadway",
    "city": "Seattle",
    "state": "WA",
    "zip": "98122",
    "latitude": 47.6114,
    "longitude": -122.3214
  },
  {
    "store_id": 3,
    "street": "2510 McKinney Ave",
    "city": "Dallas",
    "state": "TX",
    "zip": "75201",

>>> Printed 25 lines of 47 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'longitude': -122.2604}

>>>JSON Object # 0 pretty printed:

{ 'store_id': 1,
  'street': '3000 Telegraph Ave',
  'city': 'Berkeley',
  'state': 'CA',
  'zip': '94705',
  'latitude': 37.8555,
  'longitude': -122.2604}

>>>JS

In [23]:
my_read_flat_json("temp_stores_header_2.json", 25, 3)

------------------------------------
    temp_stores_header_2.json
------------------------------------
{
  "creator": "Acme Gourmet Meals",
  "timestamp": "2021-14-11 18:00:54",
  "file_name": "temp_stores_header_2.json",
  "version": "12.4.7",
  "legal": "Unauthorized use, duplication, or possession, blah, blah",
  "data": [
    {
      "store_id": 1,
      "street": "3000 Telegraph Ave",
      "city": "Berkeley",
      "state": "CA",
      "zip": "94705",
      "latitude": 37.8555,
      "longitude": -122.2604
    },
    {
      "store_id": 2,
      "street": "1001 Broadway",
      "city": "Seattle",
      "state": "WA",
      "zip": "98122",
      "latitude": 47.6114,
      "longitude": -122.3214
    },

>>> Printed 25 lines of 54 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'longitude': -122.2604}

>>>JSON Object # 0 pretty printed:

{ 'store_id': 1,
  'street

In [24]:
my_read_flat_json("temp_stores_big_data_2.json", 25, 3)

------------------------------------
    temp_stores_big_data_2.json
------------------------------------
{"store_id": 1, "street": "3000 Telegraph Ave", "city": "Berkeley", "state": "CA", "zip": "94705", "latitude": 37.8555, "longitude": -122.2604}
{"store_id": 2, "street": "1001 Broadway", "city": "Seattle", "state": "WA", "zip": "98122", "latitude": 47.6114, "longitude": -122.3214}
{"store_id": 3, "street": "2510 McKinney Ave", "city": "Dallas", "state": "TX", "zip": "75201", "latitude": 32.7958, "longitude": -96.8015}
{"store_id": 4, "street": "299 SE 3rd Ave", "city": "Miami", "state": "FL", "zip": "33131", "latitude": 25.772, "longitude": -80.1891}
{"store_id": 5, "street": "1202 Broadway", "city": "Nashville", "state": "TN", "zip": "37203", "latitude": 36.1568, "longitude": -86.7881}

>>> Printed 25 lines of 5 total lines.

>>>JSON Object # 0 unformatted:

 {'store_id': 1, 'street': '3000 Telegraph Ave', 'city': 'Berkeley', 'state': 'CA', 'zip': '94705', 'latitude': 37.8555, 'lo

## You try it - extract the following database tables into 3 separate flat json tables (no header, header, and big data); read each file to verify:

* temp_sales to temp_sales_no_header_2.json, temp_sales_header_2.json, temp_sales_big_data_2.json
* temp_line_items to temp_line_items_no_header_2.json, temp_line_items_header_2.json, temp_line_items_big_data_2.json
* temp_customers to temp_customers_no_header_2.json, temp_customers_header_2.json, temp_customers_big_data_2.json
* temp_products to temp_products_no_header_2.json, temp_products_header_2.json, temp_products_big_data_2.json
* temp_holidays to temp_holidays_no_header_2.json, temp_holidays_header_2.json, temp_holidays_big_data_2.json
